# Deadlock Detection and Prevention

A **deadlock** occurs when two or more transactions are waiting indefinitely for one another to release locks on resources. Understanding deadlocks is critical for database administrators and developers working with concurrent systems.

---

## The Four Necessary Conditions for Deadlock (Coffman Conditions)

All four conditions must hold simultaneously for a deadlock to occur:

| Condition | Description | Example |
|-----------|-------------|----------|
| **Mutual Exclusion** | At least one resource must be held in a non-shareable mode | Only one transaction can hold an exclusive lock on a row |
| **Hold and Wait** | A process holding at least one resource is waiting to acquire additional resources held by other processes | Transaction T1 holds lock on A, waiting for lock on B |
| **No Preemption** | Resources cannot be forcibly taken away from a process; they must be released voluntarily | Locks are only released when a transaction commits/aborts |
| **Circular Wait** | A circular chain of processes exists where each waits for a resource held by the next | T1 → T2 → T3 → T1 |

## Deadlock Detection with Wait-For Graph

A **Wait-For Graph (WFG)** is a directed graph used to detect deadlocks:
- **Nodes** represent transactions
- **Edges** represent "waits-for" relationships (T1 → T2 means T1 is waiting for T2)
- **Deadlock exists** if and only if the graph contains a **cycle**

Let's implement a deadlock detector using a wait-for graph:

In [ ]:
from collections import defaultdict
from typing import List, Set, Tuple, Optional


class WaitForGraph:
    """Wait-For Graph for deadlock detection in database transactions."""
    
    def __init__(self):
        self.graph = defaultdict(set)  # adjacency list: transaction -> set of transactions it waits for
        self.locks = {}  # resource -> transaction holding the lock
        self.waiting = {}  # transaction -> resource it's waiting for
    
    def acquire_lock(self, transaction: str, resource: str) -> bool:
        """Attempt to acquire a lock on a resource."""
        if resource not in self.locks:
            # Resource is free, grant the lock
            self.locks[resource] = transaction
            print(f"✅ {transaction} acquired lock on {resource}")
            return True
        
        holder = self.locks[resource]
        if holder == transaction:
            print(f"ℹ️  {transaction} already holds lock on {resource}")
            return True
        
        # Resource is held by another transaction - add wait edge
        self.waiting[transaction] = resource
        self.graph[transaction].add(holder)
        print(f"⏳ {transaction} waiting for {resource} (held by {holder})")
        return False
    
    def release_lock(self, transaction: str, resource: str):
        """Release a lock on a resource."""
        if self.locks.get(resource) == transaction:
            del self.locks[resource]
            # Remove wait edges pointing to this transaction for this resource
            for t in list(self.waiting.keys()):
                if self.waiting[t] == resource:
                    self.graph[t].discard(transaction)
                    if not self.graph[t]:
                        del self.graph[t]
                    del self.waiting[t]
            print(f"🔓 {transaction} released lock on {resource}")
    
    def detect_cycle(self) -> Optional[List[str]]:
        """Detect if there's a cycle in the wait-for graph using DFS."""
        visited = set()
        rec_stack = set()
        path = []
        
        def dfs(node: str) -> Optional[List[str]]:
            visited.add(node)
            rec_stack.add(node)
            path.append(node)
            
            for neighbor in self.graph.get(node, set()):
                if neighbor not in visited:
                    result = dfs(neighbor)
                    if result:
                        return result
                elif neighbor in rec_stack:
                    # Found a cycle - extract it
                    cycle_start = path.index(neighbor)
                    return path[cycle_start:] + [neighbor]
            
            path.pop()
            rec_stack.remove(node)
            return None
        
        for node in self.graph:
            if node not in visited:
                cycle = dfs(node)
                if cycle:
                    return cycle
        return None
    
    def check_deadlock(self) -> Tuple[bool, Optional[List[str]]]:
        """Check for deadlock and return cycle if found."""
        cycle = self.detect_cycle()
        if cycle:
            print(f"\n🔴 DEADLOCK DETECTED!")
            print(f"   Cycle: {' → '.join(cycle)}")
            return True, cycle
        print("\n🟢 No deadlock detected")
        return False, None

### Simulating a Deadlock Scenario

Let's simulate a classic deadlock scenario where:
- **T1** holds lock on Resource A, wants Resource B
- **T2** holds lock on Resource B, wants Resource A

In [ ]:
# Create a new wait-for graph
wfg = WaitForGraph()

print("=== Simulating Deadlock Scenario ===")
print()

# T1 acquires lock on Resource A
wfg.acquire_lock("T1", "Resource_A")

# T2 acquires lock on Resource B
wfg.acquire_lock("T2", "Resource_B")

# T1 tries to acquire lock on Resource B (will wait)
wfg.acquire_lock("T1", "Resource_B")

# T2 tries to acquire lock on Resource A (will wait - creates deadlock!)
wfg.acquire_lock("T2", "Resource_A")

# Check for deadlock
wfg.check_deadlock()

### More Complex Deadlock Example

Let's simulate a three-way deadlock: T1 → T2 → T3 → T1

In [ ]:
# Create a new wait-for graph for three-way deadlock
wfg3 = WaitForGraph()

print("=== Simulating Three-Way Deadlock ===")
print()

# Each transaction acquires one resource
wfg3.acquire_lock("T1", "Resource_A")
wfg3.acquire_lock("T2", "Resource_B")
wfg3.acquire_lock("T3", "Resource_C")

print()

# Now each tries to acquire the next resource (circular wait)
wfg3.acquire_lock("T1", "Resource_B")  # T1 waits for T2
wfg3.acquire_lock("T2", "Resource_C")  # T2 waits for T3
wfg3.acquire_lock("T3", "Resource_A")  # T3 waits for T1 (completes the cycle!)

# Check for deadlock
wfg3.check_deadlock()

## Visualizing the Deadlock Cycle

Let's create a visual representation of the wait-for graph to see the deadlock cycle clearly.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np


def visualize_wait_for_graph(wfg: WaitForGraph, title: str = "Wait-For Graph"):
    """Visualize the wait-for graph with deadlock cycle highlighted."""
    
    # Get all transactions
    all_transactions = set(wfg.graph.keys())
    for targets in wfg.graph.values():
        all_transactions.update(targets)
    all_transactions = sorted(all_transactions)
    
    if not all_transactions:
        print("No transactions in the graph")
        return
    
    # Detect cycle for highlighting
    cycle = wfg.detect_cycle()
    cycle_edges = set()
    if cycle:
        for i in range(len(cycle) - 1):
            cycle_edges.add((cycle[i], cycle[i + 1]))
    
    # Position nodes in a circle
    n = len(all_transactions)
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    positions = {t: (np.cos(a), np.sin(a)) for t, a in zip(all_transactions, angles)}
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    
    # Draw edges
    for source, targets in wfg.graph.items():
        for target in targets:
            x1, y1 = positions[source]
            x2, y2 = positions[target]
            
            # Check if this edge is part of the cycle
            is_cycle_edge = (source, target) in cycle_edges
            color = 'red' if is_cycle_edge else 'gray'
            linewidth = 3 if is_cycle_edge else 1.5
            
            # Draw arrow
            ax.annotate(
                '', xy=(x2 * 0.85, y2 * 0.85), xytext=(x1 * 0.85, y1 * 0.85),
                arrowprops=dict(
                    arrowstyle='->', color=color, lw=linewidth,
                    connectionstyle='arc3,rad=0.1'
                )
            )
    
    # Draw nodes
    for t, (x, y) in positions.items():
        is_in_cycle = cycle and t in cycle
        color = '#ff6b6b' if is_in_cycle else '#4dabf7'
        circle = plt.Circle((x, y), 0.15, color=color, ec='black', lw=2, zorder=10)
        ax.add_patch(circle)
        ax.text(x, y, t, ha='center', va='center', fontsize=12, fontweight='bold', zorder=11)
    
    # Add legend
    legend_elements = [
        mpatches.Patch(facecolor='#4dabf7', edgecolor='black', label='Transaction (no deadlock)'),
        mpatches.Patch(facecolor='#ff6b6b', edgecolor='black', label='Transaction (in deadlock cycle)'),
        plt.Line2D([0], [0], color='gray', lw=1.5, label='Waits-for edge'),
        plt.Line2D([0], [0], color='red', lw=3, label='Deadlock cycle edge'),
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


# Visualize the three-way deadlock
visualize_wait_for_graph(wfg3, "Three-Way Deadlock: T1 → T2 → T3 → T1")

## Deadlock Prevention Strategies

There are several strategies to prevent deadlocks:

### 1. Lock Ordering (Breaking Circular Wait)

If all transactions acquire locks in a **predefined global order**, circular wait cannot occur.

```
Rule: Always acquire locks in alphabetical/numerical order of resource names
```

In [ ]:
class OrderedLockManager:
    """Lock manager that enforces lock ordering to prevent deadlocks."""
    
    def __init__(self):
        self.locks = {}  # resource -> transaction
        self.transaction_locks = defaultdict(list)  # track lock order per transaction
    
    def acquire_locks(self, transaction: str, resources: List[str]) -> bool:
        """
        Acquire locks in sorted order to prevent deadlock.
        All-or-nothing semantics: either all locks are acquired or none.
        """
        # Sort resources to ensure consistent ordering
        sorted_resources = sorted(resources)
        acquired = []
        
        print(f"\n{transaction} requesting locks on: {resources}")
        print(f"  → Ordered acquisition: {sorted_resources}")
        
        for resource in sorted_resources:
            if resource in self.locks and self.locks[resource] != transaction:
                # Cannot acquire, release all acquired locks (rollback)
                print(f"  ❌ Cannot acquire {resource} (held by {self.locks[resource]})")
                for r in acquired:
                    del self.locks[r]
                print(f"  🔄 Rolling back: released {acquired}")
                return False
            
            self.locks[resource] = transaction
            acquired.append(resource)
            print(f"  ✅ Acquired {resource}")
        
        self.transaction_locks[transaction] = sorted_resources
        return True
    
    def release_all(self, transaction: str):
        """Release all locks held by a transaction."""
        for resource in self.transaction_locks.get(transaction, []):
            if self.locks.get(resource) == transaction:
                del self.locks[resource]
        self.transaction_locks[transaction] = []
        print(f"\n🔓 {transaction} released all locks")


# Demonstrate lock ordering preventing deadlock
print("=== Lock Ordering Prevention Demo ===")
lm = OrderedLockManager()

# Even though T1 wants B,A and T2 wants A,B, ordering ensures no deadlock
print("\nScenario: T1 wants [B, A], T2 wants [A, B]")
print("With ordering, both will try: [A, B]")

# T1 acquires in order
success1 = lm.acquire_locks("T1", ["B", "A"])  # Will be reordered to [A, B]

# T2 tries to acquire - will fail since A is held by T1
success2 = lm.acquire_locks("T2", ["A", "B"])

print(f"\nT1 success: {success1}, T2 success: {success2}")
print("→ No deadlock! T2 must wait and retry after T1 commits.")

### 2. Lock Timeout (Breaking Hold-and-Wait)

Transactions that wait too long for a lock are aborted and restarted.

In [ ]:
import time
import threading
from dataclasses import dataclass
from datetime import datetime


@dataclass
class LockRequest:
    transaction: str
    resource: str
    request_time: float


class TimeoutLockManager:
    """Lock manager with timeout-based deadlock prevention."""
    
    def __init__(self, timeout_seconds: float = 2.0):
        self.locks = {}
        self.waiting = {}  # transaction -> LockRequest
        self.timeout = timeout_seconds
        self._lock = threading.Lock()
    
    def try_acquire(self, transaction: str, resource: str) -> str:
        """Try to acquire a lock with timeout."""
        with self._lock:
            if resource not in self.locks:
                self.locks[resource] = transaction
                return "GRANTED"
            
            if self.locks[resource] == transaction:
                return "ALREADY_HELD"
            
            # Record wait time
            self.waiting[transaction] = LockRequest(transaction, resource, time.time())
            return "WAITING"
    
    def check_timeouts(self) -> List[str]:
        """Check for timed-out transactions."""
        current_time = time.time()
        timed_out = []
        
        with self._lock:
            for txn, request in list(self.waiting.items()):
                wait_time = current_time - request.request_time
                if wait_time > self.timeout:
                    timed_out.append(txn)
                    del self.waiting[txn]
        
        return timed_out


# Demonstrate timeout mechanism
print("=== Timeout-Based Prevention Demo ===")
print()

tlm = TimeoutLockManager(timeout_seconds=2.0)

# Simulate lock acquisition
print(f"T1 requests Resource_A: {tlm.try_acquire('T1', 'Resource_A')}")
print(f"T2 requests Resource_A: {tlm.try_acquire('T2', 'Resource_A')}")

print(f"\nWaiting 2.5 seconds to trigger timeout...")
time.sleep(2.5)

timed_out = tlm.check_timeouts()
print(f"\nTimed out transactions: {timed_out}")
print("→ T2 would be aborted and retried, preventing potential deadlock.")

### 3. Wait-Die and Wound-Wait Schemes

These are timestamp-based prevention schemes:

| Scheme | Older Transaction | Younger Transaction |
|--------|-------------------|---------------------|
| **Wait-Die** | Waits for younger | Dies (aborts) if waiting for older |
| **Wound-Wait** | Wounds (aborts) younger | Waits for older |

In [ ]:
class WaitDieWoundWait:
    """Implementation of Wait-Die and Wound-Wait deadlock prevention."""
    
    def __init__(self):
        self.locks = {}  # resource -> transaction
        self.timestamps = {}  # transaction -> timestamp (lower = older)
        self.timestamp_counter = 0
    
    def begin_transaction(self, transaction: str):
        """Start a new transaction with a timestamp."""
        self.timestamps[transaction] = self.timestamp_counter
        self.timestamp_counter += 1
        print(f"⏱️  {transaction} started with timestamp {self.timestamps[transaction]}")
    
    def request_lock_wait_die(self, transaction: str, resource: str) -> str:
        """
        Wait-Die: Older transactions wait, younger ones die.
        """
        if resource not in self.locks:
            self.locks[resource] = transaction
            return "GRANTED"
        
        holder = self.locks[resource]
        if holder == transaction:
            return "ALREADY_HELD"
        
        requester_ts = self.timestamps[transaction]
        holder_ts = self.timestamps[holder]
        
        if requester_ts < holder_ts:  # Requester is older
            return "WAIT"  # Older waits for younger
        else:
            return "DIE"  # Younger dies (must abort and restart)
    
    def request_lock_wound_wait(self, transaction: str, resource: str) -> str:
        """
        Wound-Wait: Older transactions wound (abort) younger, younger ones wait.
        """
        if resource not in self.locks:
            self.locks[resource] = transaction
            return "GRANTED"
        
        holder = self.locks[resource]
        if holder == transaction:
            return "ALREADY_HELD"
        
        requester_ts = self.timestamps[transaction]
        holder_ts = self.timestamps[holder]
        
        if requester_ts < holder_ts:  # Requester is older
            return f"WOUND:{holder}"  # Abort the younger holder
        else:
            return "WAIT"  # Younger waits for older


# Demonstrate Wait-Die
print("=== Wait-Die Demonstration ===")
print()
wd = WaitDieWoundWait()

# T1 is older, T2 is younger
wd.begin_transaction("T1")
wd.begin_transaction("T2")

print()
print(f"T1 requests Resource_A: {wd.request_lock_wait_die('T1', 'Resource_A')}")
print(f"T2 requests Resource_A: {wd.request_lock_wait_die('T2', 'Resource_A')}")
print("→ T2 (younger) must DIE because T1 (older) holds the lock")

print()
print("=== Wound-Wait Demonstration ===")
print()
ww = WaitDieWoundWait()

ww.begin_transaction("T1")
ww.begin_transaction("T2")

print()
print(f"T2 requests Resource_A: {ww.request_lock_wound_wait('T2', 'Resource_A')}")
print(f"T1 requests Resource_A: {ww.request_lock_wound_wait('T1', 'Resource_A')}")
print("→ T1 (older) WOUNDS T2 (younger), causing T2 to abort")

## Deadlock Resolution

When a deadlock is detected, one or more transactions must be **aborted (victim selection)**. The victim is typically chosen based on:

1. **Youngest transaction** (least work done)
2. **Transaction with fewest locks**
3. **Transaction closest to completion** (minimize wasted work)
4. **Transaction that has been rolled back the least** (prevent starvation)

In [ ]:
@dataclass
class TransactionInfo:
    name: str
    start_time: float
    locks_held: int
    work_done: int  # e.g., number of operations completed
    rollback_count: int


def select_victim(transactions: List[TransactionInfo]) -> TransactionInfo:
    """
    Select a victim for deadlock resolution.
    Uses a weighted scoring system.
    """
    print("\n📊 Victim Selection Analysis:")
    print("-" * 60)
    
    scores = []
    for txn in transactions:
        # Lower score = better victim candidate
        score = (
            -txn.start_time * 0.3 +  # Newer transactions (higher time) are better victims
            txn.locks_held * 0.2 +   # Fewer locks = better victim
            -txn.work_done * 0.3 +   # Less work done = better victim  
            txn.rollback_count * 0.2  # More rollbacks = less desirable victim (prevent starvation)
        )
        scores.append((txn, score))
        print(f"  {txn.name}: age={1000-txn.start_time:.0f}ms, locks={txn.locks_held}, "
              f"work={txn.work_done}, rollbacks={txn.rollback_count} → score={score:.2f}")
    
    # Select transaction with lowest score (best victim)
    victim = min(scores, key=lambda x: x[1])[0]
    print(f"\n🎯 Selected victim: {victim.name}")
    return victim


# Example victim selection
transactions_in_deadlock = [
    TransactionInfo("T1", start_time=100, locks_held=3, work_done=50, rollback_count=0),
    TransactionInfo("T2", start_time=500, locks_held=1, work_done=10, rollback_count=0),
    TransactionInfo("T3", start_time=800, locks_held=2, work_done=5, rollback_count=2),
]

print("=== Deadlock Resolution: Victim Selection ===")
victim = select_victim(transactions_in_deadlock)

## Summary: Prevention vs Detection

| Approach | Pros | Cons |
|----------|------|------|
| **Prevention** | No deadlocks occur | May limit concurrency, transactions may abort unnecessarily |
| **Detection** | Higher concurrency, no unnecessary aborts | Overhead of running detection algorithm, resolution cost |

In [ ]:
# Create a comparison visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Prevention strategies comparison
strategies = ['Lock\nOrdering', 'Timeouts', 'Wait-Die', 'Wound-Wait']
complexity = [2, 3, 4, 4]
concurrency = [3, 4, 3, 3]
overhead = [1, 2, 3, 3]

x = np.arange(len(strategies))
width = 0.25

bars1 = axes[0].bar(x - width, complexity, width, label='Implementation Complexity', color='#ff6b6b')
bars2 = axes[0].bar(x, concurrency, width, label='Concurrency Allowed', color='#4dabf7')
bars3 = axes[0].bar(x + width, overhead, width, label='Runtime Overhead', color='#51cf66')

axes[0].set_xlabel('Prevention Strategy')
axes[0].set_ylabel('Score (1-5)')
axes[0].set_title('Deadlock Prevention Strategies Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(strategies)
axes[0].legend()
axes[0].set_ylim(0, 6)

# Conditions breakdown
conditions = ['Mutual\nExclusion', 'Hold\nand Wait', 'No\nPreemption', 'Circular\nWait']
preventable = [1, 4, 3, 5]  # How easy to prevent (1-5)
colors = ['#868e96', '#51cf66', '#fab005', '#4dabf7']

bars = axes[1].barh(conditions, preventable, color=colors)
axes[1].set_xlabel('Ease of Prevention (1=Hard, 5=Easy)')
axes[1].set_title('Coffman Conditions: Prevention Difficulty')
axes[1].set_xlim(0, 6)

# Add value labels
for bar in bars:
    width = bar.get_width()
    axes[1].text(width + 0.1, bar.get_y() + bar.get_height()/2, 
                 f'{width}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

---

## 📌 Key Takeaways

1. **Deadlock requires ALL four Coffman conditions** — break any one to prevent deadlocks

2. **Wait-For Graph** is the standard method for deadlock detection — a cycle indicates deadlock

3. **Lock ordering** is the simplest prevention strategy — all transactions acquire locks in the same global order

4. **Timeouts** provide a practical fallback — transactions that wait too long are aborted

5. **Wait-Die/Wound-Wait** use timestamps to ensure progress — older transactions have priority

6. **Victim selection** should minimize wasted work while preventing starvation

7. **Trade-off**: Prevention limits concurrency but avoids detection overhead; Detection allows more concurrency but requires resolution

---

### Common Database Deadlock Handling

| Database | Default Strategy |
|----------|------------------|
| PostgreSQL | Detection + timeout (`deadlock_timeout` = 1s default) |
| MySQL/InnoDB | Detection (checks every 10 seconds) |
| SQL Server | Detection + victim selection (least cost) |
| Oracle | Detection + automatic rollback of one transaction |